In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import seaborn as sns
#from itertools import izip

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/unlabeled.csv')

sub_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [3]:
#train_data.groupby('hospital_death').hist()
#list_plot = list(np.arange(169,173))

In [4]:
# df = train_data.iloc[:,list_plot]
# df['hospital_death'] = train_data['hospital_death']

In [5]:
#df.groupby('hospital_death').hist()
#train_data[['height','hospital_death']].groupby('hospital_death').hist()

In [6]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [7]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [8]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

#to_label_encode = list(data_dict[data_dict['Data Type'].isin(['string'])]['Variable Name'])

In [9]:
cat_var = cat_var + ['weightclass']

In [10]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [73]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [12]:
x_test = test_data.drop('hospital_death',axis=1)

In [13]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [14]:
list_drop = ['encounter_id', 'patient_id', 'readmission_status']

In [15]:
# # Remove Features with more than 70 percent missing values
#x_comb = x_comb.drop(['readmission_status'],axis = 1)
x_comb = x_comb.drop(list_drop,axis = 1)

In [16]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [17]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))


100%|██████████| 30/30 [00:03<00:00,  7.94it/s]


In [18]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] =  x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [19]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 71848.75it/s]


In [20]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [21]:
list_cols_nans = list(data_dict[data_dict['Category'].isin(['vitals','labs','labs blood gas'])]['Variable Name'])

In [22]:
list_cols_nans = list(set(list_cols_nans).intersection(set(x_comb.columns)))
#list_cols_nans

In [23]:
x_comb1 = x_comb.copy()

In [24]:
x_comb1['unique_id'] = range(0,len(x_comb1))

In [25]:
x_comb2 = x_comb1.copy()

In [26]:
x_comb2_tr = x_comb2[x_comb2['apache_4a_hospital_death_prob'].notnull()]
x_comb2_pred = x_comb2[x_comb2['apache_4a_hospital_death_prob'].isnull()]

In [27]:
x_demo_tr = x_comb2_tr.drop('apache_4a_hospital_death_prob',axis=1)
x_demo_te = x_comb2_pred.drop('apache_4a_hospital_death_prob',axis=1)

y_demo_tr = x_comb2_tr['apache_4a_hospital_death_prob']


In [28]:
reg_params = {'boost': 'gbdt',
            'learning_rate': 0.01,
            'max_depth':6,
            'metric':{'rmsle'},
            'num_threads': -1,
            'objective': 'regression',
            'verbosity': 1,
            'num_leaves':15,
            'bagging_fraction':0.5, #rows
            'bagging_frequency':5,
            'min_data_in_leaf':50,
            'feature_fraction':0.3, # features
            'lambda_l1':10,
            'lambda_l2':15,
            'bagging_seed':2019,
            'min_gain_split':0.3
            }

In [29]:
x_train, x_valid, y_train, y_valid = train_test_split(x_demo_tr, y_demo_tr, test_size=0.1, random_state=42)

In [30]:
d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

In [31]:
lgb_clf_demo = lgb.train(reg_params,
                    d_train,
                    10000,
                    valid_sets = [d_train, d_valid],
                    verbose_eval=100
                    )

In [32]:
apache_4a_pred = lgb_clf_demo.predict(x_demo_te)

In [33]:
x_demo_te.index = range(0,len(x_demo_te))

In [34]:
dict_map = {}

for i in tqdm(range(0,len(x_demo_te))):
    u_id = x_demo_te['unique_id'].iloc[i]
    dict_map[u_id] = apache_4a_pred[i]

100%|██████████| 16550/16550 [00:00<00:00, 68276.67it/s]


In [35]:
x_comb_imp = x_comb1[['apache_4a_hospital_death_prob','unique_id']]

In [36]:
list_keys = list(dict_map.keys())

In [37]:
for i in tqdm(range(0,len(x_comb_imp))):
    
    u_id = x_comb_imp['unique_id'].iloc[i]
    
    if u_id in list_keys :
        
        ap_val = dict_map[u_id]
        
        if ap_val >0 and ap_val <1:
            
            x_comb_imp['apache_4a_hospital_death_prob'].iloc[i] = ap_val
        
#         elif ap_val<0:
#             x_comb_imp['apache_4a_hospital_death_prob'].iloc[i] = 0
        
#         elif ap_val>0:
#             x_comb_imp['apache_4a_hospital_death_prob'].iloc[i] = 1
        
        
            
            


  0%|          | 0/131021 [00:00<?, ?it/s]/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 131021/131021 [00:46<00:00, 2836.90it/s]


In [38]:
x_comb['apache_4a_hospital_death_prob'] = x_comb_imp['apache_4a_hospital_death_prob']

In [39]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

30it [00:01, 16.92it/s]


In [40]:
#List to be biined -----> VITALS
#to_binned_columns = list(data_dict[data_dict['Category'] == 'labs blood gas']['Variable Name'].unique())
to_binned_columns = list(data_dict[data_dict['Category'] == 'labs']['Variable Name'].unique())
to_binned_columns = list(set(to_binned_columns).intersection(set(x_comb.columns)))

In [41]:
binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10 ,labels=False))
    binned_cols.append(cols+'_binned')
    

100%|██████████| 60/60 [00:01<00:00, 40.28it/s]


In [42]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 8,labels=False))

In [43]:
featured_cols = binned_cols + ['age_category','height'+'_binned']

In [44]:
#2way interaction
bias_cols = ['gender','age']

In [45]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

100%|██████████| 2/2 [00:14<00:00,  7.46s/it]


In [46]:
#Critical -3way interaction


x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)
x_comb['age'+'gender'+'ethnicity' +'weighted_classt' ] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str) +'_' + x_comb['weightclass'].astype(str)


inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt'] +['age'+'gender'+'ethnicity'] + ['age'+'gender'+'ethnicity'+'weighted_classt']

In [47]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

100%|██████████| 123/123 [00:16<00:00,  7.32it/s]


In [49]:
list_illness = ['aids',
                'cirrhosis',
                'diabetes_mellitus',
                'hepatic_failure',
                'immunosuppression',
                'leukemia',
                'lymphoma',
                'solid_tumor_with_metastasis']


In [50]:
inter_cols1 = []

for col in tqdm(list_illness) :
    x_comb['hospital_id'+str(col)] = x_comb['hospital_id'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['gender'+str(col)] = x_comb['gender'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['age'+str(col)] = x_comb['age'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['bmi'+str(col)] = x_comb['bmi'].astype(str) + '_' + x_comb[col].astype(str)
    
    inter_cols1.append('hospital_id'+str(col))
    inter_cols1.append('gender'+str(col))
    inter_cols1.append('age'+str(col))
    inter_cols1.append('bmi'+str(col))

100%|██████████| 8/8 [00:05<00:00,  1.58it/s]


In [51]:
for cols in tqdm(inter_cols1):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 32/32 [00:01<00:00, 23.97it/s]


In [52]:
inter_cols2 = []

for col in tqdm(list_illness) :
    x_comb['age_category'+str(col)] = x_comb['age_category'].astype(str) + '_' + x_comb[col].astype(str)
    inter_cols2.append('age_category'+str(col))

100%|██████████| 8/8 [00:01<00:00,  6.45it/s]


In [53]:
for cols in tqdm(inter_cols2):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 8/8 [00:00<00:00, 34.64it/s]


In [90]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']


In [91]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    



0it [00:00, ?it/s]

4it [00:00, 33.82it/s]

8it [00:00, 35.15it/s]

13it [00:00, 37.26it/s]

18it [00:00, 39.16it/s]

22it [00:00, 39.11it/s]

30it [00:00, 42.35it/s]


In [56]:
# aggs = {}
# aggs['apache_4a_hospital_death_prob'] = ['median', 'mean', 'max', 'min']
# aggs['apache_4a_icu_death_prob'] = ['median', 'mean', 'max', 'min']


In [78]:
new_cat_var = list(set(new_cat_var).intersection(x_comb.columns))

In [81]:
# for col in tqdm(new_cat_var):
    
#     aggs_temp = aggs.copy()
#     #aggs_temp.pop(col)
    
#     agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
#     agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
#     x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
#                     left_on=[col], right_on=[col])
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = x_comb['apache_4a_hospital_death_prob'].mean()
    
    #Compute the number of values and the mean of each group
    agg = x_comb.groupby(col)['apache_4a_hospital_death_prob'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    



0it [00:00, ?it/s]

3it [00:00,  9.71it/s]

4it [00:00,  6.43it/s]

5it [00:00,  5.17it/s]

6it [00:01,  4.50it/s]

7it [00:01,  4.13it/s]

8it [00:01,  3.97it/s]

9it [00:02,  3.83it/s]

10it [00:02,  3.77it/s]

11it [00:02,  3.76it/s]

12it [00:02,  3.64it/s]

13it [00:03,  3.49it/s]

14it [00:03,  3.48it/s]

15it [00:03,  3.41it/s]

16it [00:04,  3.38it/s]

17it [00:04,  3.44it/s]

18it [00:04,  3.46it/s]

19it [00:04,  3.51it/s]

20it [00:05,  3.52it/s]

21it [00:05,  3.60it/s]

22it [00:05,  3.42it/s]

23it [00:06,  3.39it/s]

24it [00:06,  3.33it/s]

25it [00:06,  3.36it/s]

26it [00:07,  3.22it/s]

27it [00:07,  3.29it/s]

28it [00:07,  3.23it/s]

29it [00:07,  3.22it/s]

30it [00:08,  3.63it/s]


In [59]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [61]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [63]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [64]:
def mode(x):
    return Counter(x).most_common(1)[0][0]


In [65]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count',mode]
    

100%|██████████| 30/30 [00:00<00:00, 61711.19it/s]


In [66]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [68]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])


100%|██████████| 30/30 [07:40<00:00, 15.36s/it]


In [69]:
x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

le = LabelEncoder()
x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [70]:
#x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [71]:
y_train.shape

(103023,)

In [92]:
train_x = x_comb[0:91713]
test_x =  x_comb[91713:]

In [93]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [94]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':6,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':15,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.2, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [95]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [96]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    13000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=1500,
                    verbose_eval=200
                    )

Training until validation scores don't improve for 1500 rounds.
[200]	training's auc: 0.896669	valid_1's auc: 0.886036
[400]	training's auc: 0.906778	valid_1's auc: 0.893552
[600]	training's auc: 0.913055	valid_1's auc: 0.897568
[800]	training's auc: 0.917567	valid_1's auc: 0.899968
[1000]	training's auc: 0.921141	valid_1's auc: 0.901554
[1200]	training's auc: 0.924223	valid_1's auc: 0.902784
[1400]	training's auc: 0.926952	valid_1's auc: 0.903727
[1600]	training's auc: 0.929499	valid_1's auc: 0.90442
[1800]	training's auc: 0.931761	valid_1's auc: 0.904922
[2000]	training's auc: 0.933889	valid_1's auc: 0.90541
[2200]	training's auc: 0.935883	valid_1's auc: 0.905704
[2400]	training's auc: 0.937784	valid_1's auc: 0.905953
[2600]	training's auc: 0.939521	valid_1's auc: 0.906172
[2800]	training's auc: 0.941179	valid_1's auc: 0.906382
[3000]	training's auc: 0.942724	valid_1's auc: 0.906483
[3200]	training's auc: 0.944305	valid_1's auc: 0.906672
[3400]	training's auc: 0.945841	valid_1's auc:

KeyboardInterrupt: 

In [87]:
predictions = lgb_clf.predict(test_x)

In [88]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [89]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/apache_imputed_23rdfeb_canbeoverfit.csv',index = False)


In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
list_unimp = list(feature_imp[feature_imp['Value'] == 0]['Feature'])

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=130, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()
# #data_sub

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 20000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 1500)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

In [ ]:
predictions_10 = predictions /10.0
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions_10

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/best_till_now_imputing_apache_20thfeb.csv',index = False)


In [ ]:
#feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
#feature_imp[feature_imp['Feature'] == 'weightclass']


In [ ]:
#feature_imp[feature_imp['Value'] == 0]

In [ ]:
a1 =  pd.read_csv('/Users/s0c02nj/Desktop/sub_10fold_keep_missing_as_it_is_18thFeb2020_newfeat.csv')
a2 =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_12thfeb.csv')

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4_lgbold_10thfeb_new_bmi_impute.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_10thFeb2020_lgb_type4_ff_20_na_bmi.csv')
# #a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb_type3_ff_27.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_type1_type2_ensemble_5thFeb2020.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_11thFeb2020_lgb_type3_ff_27.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/pred_lgb_keeping_cat_cont_interchanged_11th_feb.csv')
# a6 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4tillnpw_11thfeb.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (a1['hospital_death'] + a2['hospital_death']
           )/2.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_18thfeb.csv',index = False)